In [1]:
import random
from sklearn import svm
from sklearn.model_selection import train_test_split
# human = label 0
# AI = label 1
import pandas as pd
import json
import os
import numpy as np
from sklearn.model_selection import KFold
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import math

foldername = './Data_all/AIembeddings/'
human_embeddings = np.load(foldername + 'human_embeddings_curie.npy')
ofa_embeddings = np.load(foldername + 'ofa_embeddings_curie.npy')
git_embeddings = np.load(foldername + 'git_embeddings_curie.npy')
clipcap_embeddings = np.load(foldername + 'clipcap_embeddings_curie.npy')
blip_embeddings = np.load(foldername + 'blip_embeddings_curie.npy')
microsoft_api_embeddings = np.load(foldername + 'microsoft_api_embeddings_curie.npy')

ofa_sampled_idx = random.sample(range(0, 1000), 400)
git_sampled_idx = random.sample(range(0, 1000), 400)
clipcap_sampled_idx = random.sample(range(0, 1000), 400)
blip_sampled_idx = random.sample(range(0, 1000), 400)
microsoft_api_sampled_idx = random.sample(range(0, 1000), 400)

embed = []
labels = []
modelname = []

for embedding in human_embeddings:
    embed.append(embedding)
    labels.append(0)
    modelname.append('human')
    
for idx in ofa_sampled_idx:
    embed.append(ofa_embeddings[idx])
    labels.append(1)
    modelname.append('ofa')
    
for idx in git_sampled_idx:
    embed.append(git_embeddings[idx])
    labels.append(1)
    modelname.append('git')
    
for idx in clipcap_sampled_idx:
    embed.append(clipcap_embeddings[idx])
    labels.append(1)
    modelname.append('clipcap')
    
for idx in blip_sampled_idx:
    embed.append(blip_embeddings[idx])
    labels.append(1)
    modelname.append('blip')
    
for idx in microsoft_api_sampled_idx:
    embed.append(microsoft_api_embeddings[idx])
    labels.append(1)
    modelname.append('microsoft_api')
    
embed = np.vstack(embed)
print(embed.shape)
labels = np.array(labels)

(4000, 4096)


In [ ]:
print(embed.shape)
print(len(labels))
print(len(modelname))
print(type(modelname))

featuredim = embed.shape[1]
print(featuredim)

unique_elements = list(set(modelname))
print(unique_elements)

def find_indices(lst, element):
    indices = [index for index, value in enumerate(lst) if value == element]
    return indices


#########################################################################################################
############################### LEAVE one model out; train on the rest and test on the one
Dict_model_ind={}
ResultMat = []

for model in unique_elements:
    
    indicesA = [i for i, elem in enumerate(modelname) if (elem != model) and (elem != 'human')]
    indicesB = [i for i, elem in enumerate(modelname) if (elem == 'human')]
    # Determine the number of elements to choose (half of the list)
    num_elements_to_choose = len(indicesB) // 2
    # Randomly shuffle the list
    random.shuffle(indicesB)

    # Select the first half of the shuffled list
    selected_elements_train_human = indicesB[:num_elements_to_choose]
    selected_elements_test_human = indicesB[num_elements_to_choose:]
    
    indicesT = find_indices(modelname, model) 
    Dict_model_ind[model] = indicesT + selected_elements_test_human        #test set
    Dict_model_ind[model + '_opp'] = indicesA + selected_elements_train_human #train set
    #print(len(indices))
    #print(indices)

#print(Dict_model_ind)


for targetmodel in unique_elements:
    
    
    if targetmodel == 'human':
        continue
    else:
        
        result = {}
        ################# LEAVE one more out; train on the rest; and test on the remaining one model
        print('leaving: ' + targetmodel)  
        result['test'] = [targetmodel, 'human']
        print('testing set.....')
        ind_test = Dict_model_ind[targetmodel]
        #print(ind_test)
        X_test = embed[ind_test, :]
        #print(type(X_test))
        y_test = labels[ind_test]
        #print(modelname[ind_test])

        print('training set.....')
        #print(rest)
        ind_train = Dict_model_ind[targetmodel + '_opp']                
        X_train = embed[ind_train,:]
        #print(X_train.shape)
        y_train = labels[ind_train]
        #print(y_train)
        selectedMname = [modelname[i] for i in ind_train]
        allTrainModels = list(set(selectedMname))
        result['train'] = allTrainModels
         
        # Create an SVM classifier
        clf = SVC(kernel='rbf')
        
#         print(X_train.shape)
#         print(y_train.shape)
        #print(y_train)
#         nan_count = np.sum(np.isnan(y_train))

#         print("Number of NaN values in y_train:", nan_count)

        # Train the classifier on the training data
        clf.fit(X_train, y_train)

        # Make predictions on the test data
        y_pred = clf.predict(X_test)
        #print(y_pred)
        #print(y_test)

        # Calculate the accuracy of the classifier
        accuracies = (y_test == y_pred).astype(float)

        # Calculate average accuracy
        accuracy = np.mean(accuracies)

        # Calculate standard deviation of accuracy
        std_deviation = np.std(accuracies)/math.sqrt(len(accuracies))

        print("Average Accuracy:", accuracy)
        print("Standard Deviation of Accuracy:", std_deviation)

        result['accu'] = accuracy
        result['std'] = std_deviation
        result['flag'] = 'test'

        ResultMat.append(result)
        
        ############################### do the reverse; train on single; test on multiple   
        print('reversing the training and testing set')       
        result = {} 
        # Create an SVM classifier
        clf = SVC(kernel='rbf')
        
        # Train the classifier on the training data
        clf.fit(X_test, y_test)

        # Make predictions on the test data
        y_pred = clf.predict(X_train)
        
        # Calculate the accuracy of the classifier
        # Calculate the accuracy of the classifier
        accuracies = (y_train == y_pred).astype(float)
        #print(accuracies)
        # Calculate average accuracy
        accuracy = np.mean(accuracies)

        # Calculate standard deviation of accuracy
        std_deviation = np.std(accuracies)/math.sqrt(len(accuracies))

        print("Average Accuracy:", accuracy)
        print("Standard Deviation of Accuracy:", std_deviation)
                
        result['train'] = [targetmodel, 'human']
        result['test'] = allTrainModels
        result['accu'] = accuracy
        result['std'] = std_deviation
        result['flag'] = 'train'

        ResultMat.append(result)
        
print(ResultMat)

(4000, 4096)
4000
4000
<class 'list'>
4096
['ofa', 'microsoft_api', 'blip', 'clipcap', 'git', 'human']
leaving: ofa
testing set.....
training set.....
Average Accuracy: 0.6471428571428571
Standard Deviation of Accuracy: 0.012771312148490031
reversing the training and testing set
Average Accuracy: 0.39692307692307693
Standard Deviation of Accuracy: 0.009595174025940054
leaving: microsoft_api
testing set.....
training set.....
Average Accuracy: 0.6278571428571429
Standard Deviation of Accuracy: 0.012918772360417672
reversing the training and testing set
Average Accuracy: 0.4169230769230769
Standard Deviation of Accuracy: 0.009669504498721563
leaving: blip
testing set.....
training set.....


In [ ]:
### plots for grouping demographic information
def plot_res_demographics(plotname, ResultMat):
    
    import matplotlib.pyplot as plt
    import numpy as np
    from matplotlib.patches import Rectangle
    
    modelist = ['microsoft_api', 'blip', 'git', 'ofa', 'clipcap']
        
    for condition in ['train','test']:
        top1 = []
        top1_std = []
        for selectedM in modelist:
            for result in ResultMat:
                if result['flag'] == condition:
                    if selectedM in result[condition]:
#                         print(result['flag'])
#                         print(condition)
#                         print(result[condition])
                        #chancenum = 1/(len(result['test']))
                        top1.append(result['accu'])
                        top1_std.append(result['std'])
                
        print(top1)        
        ######### everything is ready for plotting                
        years = list(range(1, len(modelist)))
        strtask = modelist
        # Figure Size
        fig, ax = plt.subplots(layout='constrained')
        #x = np.arange(len(strtask))  # the label locations

        width = 0.2  # the width of the bars
        print(len(strtask))
        ax.bar(strtask, top1, yerr=top1_std, color ='black', width = 0.3,align='center',
       alpha=0.5,
       ecolor='black',
       capsize=8)   

        #ax.set_xticks(x, strtask)
        #ax.legend(loc='upper left', ncols=len(condition_chosen_list))

        plt.ylabel('Top-1 Accuracy')
        plt.xticks(rotation = 90) # Rotates X-Axis Ticks by 45-degrees

        plt.ylim(0.25, 1)
        y_ticks = [0.3,0.5,0.7,0.9]
        ax.set_yticks(y_ticks)
        #plt.xlim(0.5, len(years)+0.5)

        #draw chance 0.5 
        chancex =np.arange(len(years)+2)-0.5
        plt.plot(chancex, np.arange(len(chancex))*0+0.5, 'k--')#, label='chance (50%)')
        #plt.legend()

        fig.tight_layout()

        fig.set_figwidth(7)
        fig.set_figheight(5)

        #fig.legend(loc='upper center', bbox_to_anchor=(0.55, 1.0), ncol=3, fancybox=True, shadow=True,frameon=False)

        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)

        plt.savefig(plotname + '_' + condition + '.eps', format='eps', bbox_inches='tight')
        plt.savefig(plotname + '_' + condition + '.png',  bbox_inches='tight')
        plt.show()
#     fig.savefig('../' + plotname + '.png', bbox_inches='tight')
#     fig.savefig('Human_judge_bars_Giorgia'+condition_chosen+'.pdf', bbox_inches='tight')
    
#     fig.savefig('Human_judge_bars_Giorgia'+condition_chosen+'.png', bbox_inches='tight')

#======================AGE=======
#condition_list = ['ageL35','ageG35L45','ageG45','F','M','school','bachelor','postgrad','US','nonUS','wGenderQ','woGenderQ']
key = 'AIjudge'
plotpathname = './plots/Task1_' + key + '_TrainTestSplits_BarPlotOverall' #for saving the plots
facebarcolor = ['dimgray','darkgrey','lightgrey']

plot_res_demographics(plotpathname, ResultMat)